# モンテカルロ着地点シミュレーション

発射角度・風速・機体質量をランダムに変化させて、着地点のばらつきを統計的に評価する。

In [1]:
import math
from random import gauss

import jpype
import numpy as np

import orhelper

# Java 17+ 用: --add-opens オプションを追加
_original_startJVM = jpype.startJVM

def _patched_startJVM(*args, **kwargs):
    add_opens = [
        "--add-opens=java.base/java.lang=ALL-UNNAMED",
        "--add-opens=java.base/java.lang.reflect=ALL-UNNAMED",
        "--add-opens=java.base/java.util=ALL-UNNAMED",
    ]
    _original_startJVM(*args, *add_opens, **kwargs)

jpype.startJVM = _patched_startJVM

In [2]:
# ヘルパークラス・関数の定義

METERS_PER_DEGREE_LATITUDE = 111325
METERS_PER_DEGREE_LONGITUDE_EQUATOR = 111050


def range_flat(start, end):
    dy = (end.getLatitudeDeg() - start.getLatitudeDeg()) * METERS_PER_DEGREE_LATITUDE
    dx = (end.getLongitudeDeg() - start.getLongitudeDeg()) * METERS_PER_DEGREE_LONGITUDE_EQUATOR
    return math.sqrt(dy * dy + dx * dx)


def bearing_flat(start, end):
    dy = (end.getLatitudeDeg() - start.getLatitudeDeg()) * METERS_PER_DEGREE_LATITUDE
    dx = (end.getLongitudeDeg() - start.getLongitudeDeg()) * METERS_PER_DEGREE_LONGITUDE_EQUATOR
    return math.pi / 2 - math.atan(dy / dx)


class LandingPoint(orhelper.AbstractSimulationListener):
    def __init__(self, ranges, bearings):
        self.ranges = ranges
        self.bearings = bearings

    def endSimulation(self, status, simulation_exception):
        worldpos = status.getRocketWorldPosition()
        conditions = status.getSimulationConditions()
        launchpos = conditions.getLaunchSite()
        geodetic_computation = conditions.getGeodeticComputation()

        if geodetic_computation != geodetic_computation.FLAT:
            raise Exception("GeodeticComputationStrategy type not supported")

        self.ranges.append(range_flat(launchpos, worldpos))
        self.bearings.append(bearing_flat(launchpos, worldpos))


class AirStart(orhelper.AbstractSimulationListener):
    def __init__(self, altitude):
        self.start_altitude = altitude

    def startSimulation(self, status):
        position = status.getRocketPosition()
        position = position.add(0.0, 0.0, self.start_altitude)
        status.setRocketPosition(position)

In [3]:
instance = orhelper.OpenRocketInstance()
instance.__enter__()
orh = orhelper.Helper(instance)

doc = orh.load_doc('simple.ork')
sim = doc.getSimulation(0)

22:39:52.663 [main] INFO  n.s.o.s.providers.TranslatorProvider - Using default locale ja_JP
22:39:52.677 [main] INFO  n.s.o.s.providers.TranslatorProvider - Set up translation for locale ja_JP, debug.currentFile=messages_ja.properties
22:39:52.682 [DatabaseLoadingThread] INFO  n.s.o.d.ComponentPresetDatabaseLoader - Loading component presets from datafiles/presets
22:39:52.683 [DatabaseLoadingThread] DEBUG net.sf.openrocket.util.JarUtil - Found jar file using codeSource
22:39:52.683 [DatabaseLoadingThread] INFO  n.s.o.database.MotorDatabaseLoader - Starting reading serialized motor database
22:39:52.683 [DatabaseLoadingThread] DEBUG net.sf.openrocket.util.JarUtil - Found jar file using codeSource
22:39:52.686 [DatabaseLoadingThread] DEBUG n.s.o.database.MotorDatabaseLoader - Reading motors from file datafiles/thrustcurves/thrustcurves.ser
22:39:52.812 [DatabaseLoadingThread] INFO  n.s.o.database.MotorDatabaseLoader - Ending reading serialized motor database, motorCount=1721
22:39:52.81

In [4]:
# モンテカルロシミュレーション実行
num = 20
ranges = []
bearings = []

opts = sim.getOptions()
rocket = opts.getRocket()

for p in range(num):
    print(f'Running simulation {p}')

    opts.setLaunchRodAngle(math.radians(gauss(45, 5)))       # 45 +- 5 deg
    opts.setLaunchRodDirection(math.radians(gauss(0, 5)))     # 0 +- 5 deg
    opts.setWindSpeedAverage(gauss(15, 5))                    # 15 +- 5 m/s
    for component_name in ('Nose cone', 'Body tube'):         # 質量 +-5%
        component = orh.get_component_named(rocket, component_name)
        mass = component.getMass()
        component.setMassOverridden(True)
        component.setOverrideMass(mass * gauss(1.0, 0.05))

    airstarter = AirStart(gauss(1000, 50))                   # 高度 1000 +- 50 m
    lp = LandingPoint(ranges, bearings)
    orh.run_simulation(sim, listeners=(airstarter, lp))

Running simulation 0
Running simulation 1
Running simulation 2
Running simulation 3
Running simulation 4
Running simulation 5
Running simulation 6
Running simulation 7
Running simulation 8
Running simulation 9
Running simulation 10
Running simulation 11
Running simulation 12
Running simulation 13
Running simulation 14
Running simulation 15
Running simulation 16
Running simulation 17
Running simulation 18
Running simulation 19


In [5]:
# 結果表示
print(
    'Rocket landing zone %3.2f m +- %3.2f m bearing %3.2f deg +- %3.4f deg from launch site. Based on %i simulations.'
    % (np.mean(ranges), np.std(ranges), np.degrees(np.mean(bearings)),
       np.degrees(np.std(bearings)), num)
)

Rocket landing zone 4429.12 m +- 1366.27 m bearing 90.00 deg +- 0.0000 deg from launch site. Based on 20 simulations.


In [6]:
# JVMをシャットダウン (カーネル再起動前に実行)
instance.__exit__(None, None, None)